### **Задание №1**. Предположим, что у вас в базе есть табличка с балансом по всем счетам клиента на конец дня. Из-за особенностей построения таблицы, часть значений незаполнена и физически это означает отсутствие информации об изменении баланса. Для простоты будем считать, пропущенные значения для одного пользователя не могут встретиться между двумя заполненными. Напишите SQL запрос для заполнения неизвестных значений по каждому клиенту последним известным.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | NaN
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | NaN
3 | 18.04.2020 | NaN


In [ ]:
select client_id, date, value,
coalesce(value, first_value(value) over (partition by client_id order by date)) as value_without_null
from ...

### **Задание №1.1 (*)**. В прошлом задании мы считали, что пропущенные значения не могут находиться между двумя заполненными в рамках одного пользователя, что не всегда правда в реальных задачах. Напишите теперь SQL запрос без указанного допущения.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | 7000
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | 49000
3 | 18.04.2020 | NaN

In [ ]:
select client_id, date, value,
coalesce(value, first_value(value) over (partition by client_id order by date)) as value_without_null
from ...

### **Задание №2**. Пусть клиенты обладают рядом статусов, закодированных цифрами: 0 - "улица", 1 - "зарплатник", 2 - "премиум", 3 - "пенсионер", 4 - "студент". У нас есть матрица с логами смен статусов за некоторый период, разверните ее в матрицу статус - статус с частотами перехода между ними.

Пример, как было:

client_id | prev_segment | new_segment|
--- | --- | ---
1 | 0 | 1
2 | 0 | 2
3 | 4 | 3

Пример, как нужно:

segment |0 | 1 | 2 | 3 | 4 
--- | --- | --- | --- | --- | ---
0 | 0 | 0.5 | 0.5 | 0 | 0 
1 | 0 |0 | 0 | 0 | 0
2 | 0 | 0 | 0 | 0 | 0
3 | 0 | 0 | 0 | 0 | 0
4 | 0 | 0 | 0 | 1 | 0

В процессе выполнения задания разрешается пользоваться библиотеками numpy и pandas.

In [1]:
import numpy as np
import pandas as pd

# Сгенерируем данные для задачи
data = pd.DataFrame(
    np.random.randint(low=0, high=4, size=(50000000, 2)),
    columns=['prev_segment', 'new_segment']
)

# <YOUR_CODE_HERE>

### **Задание №2.1 (*)**. Если вы не пользовались библиотечными функциями и реализовывали алгоритм самостоятельно, напишите, пожалуйста, его асимптотическую сложность в терминах O-большое.

In [2]:
# Создаем DataFrame

df = pd.DataFrame(columns=['id', 'prev_segment', 'new_segment'])

df['id'] = data['new_segment'].sort_values().unique()
df['prev_segment'] = data['prev_segment'].sort_values().unique()
df['new_segment'] = data['new_segment'].sort_values().unique()

# считаем кол-во каждого из значений по каждой группе
counts = data.groupby('prev_segment')['new_segment'].value_counts()

# Считаем размер каждой группы для нормирования
size = data.groupby('prev_segment').count().iloc[:, 0]

# Нормируем
nn = counts / size

# Записываем в DF, пропуски заполняем нулями (при малом кол-ве данных)
result = nn.unstack().fillna(0.0)

# Делаем названия столбцов как в примере
result.index.name = None
result.columns.name = 'segment'

result

segment,0,1,2,3
0,0.250127,0.250059,0.249898,0.249916
1,0.249892,0.249865,0.250156,0.250088
2,0.250066,0.249950,0.250049,0.249935
3,0.249951,0.249974,0.249987,0.250088


### **Задание №3**. Постройте прогнозую модель для датасета с UCI Machine Learning: предобработайте категориальные фичи, отберите их, соберите схему валидации и отберите модели с помощью нее. Описание датасета по ссылке: https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Данная задача multilabel multiclass classification, поэтому предлагаю начать с одного из подходов:
- Свести задачу к multilabel binary classification, как это указано в описании к датасету;
- Свести задачку к binary classification тем или иным способом;
- Обучить множество multiclass/binary classification моделей;
- Что-то еще, что придет в голову.


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data

--2021-04-20 07:39:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346695 (339K) [application/x-httpd-php]
Saving to: ‘drug_consumption.data’

drug_consumption.da 100%[===================>] 338.57K  1.28MB/s    in 0.3s    

2021-04-20 07:39:48 (1.28 MB/s) - ‘drug_consumption.data’ saved [346695/346695]



In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('drug_consumption.data', header=None, index_col=0)
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [ ]:
# Импортируем нужные нам библиотеки

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")
RAND = 10

# Записываем датасет в переменную 
df = pd.read_csv('drug_consumption.data', header=None, index_col=0)

# отобразим все столбцы в таблице
pd.set_option('display.max_columns', None)

# Закодируем категориальные признаки
columns = [x for x in range(13, 32)]

for column in columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

# Сводим задачу к бинарной классификации
# 0 - за последнии 10 лет человек ничего не употреблял из списка
# 1 - за последнии 10 лет человек употреблял минимум одну категорию из списка
df['target'] = 0

def target(data: pd.DataFrame):
    
    for i in range(len(data)):
        if (2 or 3 or 4 or 5 or 6 or 7) in data[columns].iloc[i, :].to_list():
            data['target'].iloc[i] = 1
        else:
            data['target'].iloc[i] = 0
            
target(df)

# Разбиваем данные на train и test
X = df.drop(['target'], axis=1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, shuffle=True, random_state=RAND)

# Создаем DF для записи результатов моделей
metrics = pd.DataFrame(
    index=('ROC-AUC', 'F1', 'Accuracy', 'Precision', 'Recall'))

# Обучаем N базовых алгоритмов

# KNN
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

y_proba = knn.predict_proba(X_test)
y_pred = knn.predict(X_test)

metrics['KNeighborsClassifier'] = ((roc_auc_score(y_test, y_proba[:, 1:]),
                                  f1_score(y_test, y_pred),
                                  accuracy_score(y_test, y_pred),
                                  precision_score(y_test, y_pred),
                                  recall_score(y_test, y_pred)))

print(f'ROC-AUC = {roc_auc_score(y_test, y_proba[:, 1:])}')
print(f'F1 = {f1_score(y_test, y_pred)}')
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')

# LogisticRegression
lg = LogisticRegression(class_weight='balanced')
lg.fit(X_train, y_train)

y_proba = lg.predict_proba(X_test)
y_pred = lg.predict(X_test)

metrics['LogisticRegression'] = ((roc_auc_score(y_test, y_proba[:, 1:]),
                                  f1_score(y_test, y_pred),
                                  accuracy_score(y_test, y_pred),
                                  precision_score(y_test, y_pred),
                                  recall_score(y_test, y_pred)))

print(f'ROC-AUC = {roc_auc_score(y_test, y_proba[:, 1:])}')
print(f'F1 = {f1_score(y_test, y_pred)}')
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')

# CatBoostClassifier
cat = CatBoostClassifier(auto_class_weights='Balanced', verbose=False)
cat.fit(X_train, y_train)

y_proba = cat.predict_proba(X_test)
y_pred = cat.predict(X_test)

metrics['CatBoostClassifier'] = ((roc_auc_score(y_test, y_proba[:, 1:]),
                                  f1_score(y_test, y_pred),
                                  accuracy_score(y_test, y_pred),
                                  precision_score(y_test, y_pred),
                                  recall_score(y_test, y_pred)))

print(f'ROC-AUC = {roc_auc_score(y_test, y_proba[:, 1:])}')
print(f'F1 = {f1_score(y_test, y_pred)}')
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')

# LGBMClassifier
lgbm = LGBMClassifier(class_weight='balanced', verbose=0)
lgbm.fit(X_train, y_train)

y_proba = lgbm.predict_proba(X_test)
y_pred = lgbm.predict(X_test)

metrics['LGBMClassifier'] = ((roc_auc_score(y_test, y_proba[:, 1:]),
                                  f1_score(y_test, y_pred),
                                  accuracy_score(y_test, y_pred),
                                  precision_score(y_test, y_pred),
                                  recall_score(y_test, y_pred)))

print(f'ROC-AUC = {roc_auc_score(y_test, y_proba[:, 1:])}')
print(f'F1 = {f1_score(y_test, y_pred)}')
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')

# Выводим таблицу с результатами
metrics.style.highlight_max(color='orange', axis=1)

# Делаем кросс-валидацию для лучшего базового алгоритма
folds = StratifiedShuffleSplit(n_splits=3, test_size=0.25)
scores = cross_val_score(cat, X, y, scoring='roc_auc', cv=folds)

print(f"ShuffleSplit = {np.mean(np.abs(scores))}")

### **Задание №4**. Предположим, что мы построили рекомендательную систему, и ожидаем, что средний чек клиента вследствие этого должен вырасти. Мы провели AB-тест и собрали данные по контрольной и тестовой группе. Предложите и проведите статистический тест, который можно применить к этим данным.

В процессе выполнения задания разрешено пользоваться библиотеками scipy и statsmodels.

In [18]:
import numpy as np

# Сгенерируем данные для задачи
control = np.random.lognormal(mean=5, sigma=2, size=(100000,))
test = np.random.lognormal(mean=5.04, sigma=2, size=(100000,))

In [19]:
from scipy import stats

# Тк распределение нормальное + довольно много данных => используем двухвыборочный
# параметрический критерий t-стьюдента для зависимых выборок

print(stats.ttest_rel(control, test))
print(f"Control = {np.mean(control)}")
print(f"Test = {np.mean(test)}")

Ttest_relResult(statistic=-2.4968570435725885, pvalue=0.01253154144653884)
Control = 1099.7963423971344
Test = 1181.1925346492321


Случайность результата или p-value берем как 5%. На нашем тесте pvalue < 5% так что можем подтведтвердить альтернативную гипотезу (H1)

### **Задание №4 (*)**. В прошлом задании мы улавливали статистически значимое различие в средних в 0.04 единицы на выборке из 100000 клиентов. Насколько больше нам требуется собрать клиентов, чтобы уловить различие в 0.02?

In [ ]:
import numpy as np

# Сгенерируем данные для задачи
control = np.random.lognormal(mean=5, sigma=2, size=(600000,))
test = np.random.lognormal(mean=5.02, sigma=2, size=(600000,))

In [13]:
print(stats.ttest_rel(control, test))
print(f"Control = {np.mean(control)}")
print(f"Test = {np.mean(test)}")

Ttest_relResult(statistic=-2.060486997555255, pvalue=0.0393524377180216)
Control = 1095.3046868127574
Test = 1124.5650335535306


Примерное в 5-6 раз больше